<a href="https://colab.research.google.com/github/Bryan-Az/RL-DQN-Gym/blob/main/qlearning_project_drafts/RL_Project_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a Q Learning Agent in Gymnasium's Blackjack Environment

## Imports and Installs

In [ ]:
!pip install gym gym[atari] gym[accept-rom-license] agilerl accelerate>=0.21.0

In [ ]:
import os
import imageio
import gymnasium as gym
import numpy as np

### These imports will be used to implement the NN Agent ##
#import torch
#from agilerl.algorithms.td3 import TD3
#from agilerl.components.replay_buffer import ReplayBuffer
#from agilerl.hpo.mutation import Mutations
#from agilerl.hpo.tournament import TournamentSelection
#from agilerl.training.train_off_policy import train_off_policy
#from agilerl.utils.utils import create_population, make_vect_envs

from tqdm import tqdm
from __future__ import annotations
from collections import defaultdict

In [ ]:
env = gym.make("Blackjack-v1", render_mode="rgb_array")
n_episodes = 50_000
env = gym.wrappers.RecordEpisodeStatistics(env, deque_size=n_episodes)

In [ ]:
class BlackjackAgent:
    def __init__(
        self,
        env,
        learning_rate: float,
        initial_epsilon: float,
        epsilon_decay: float,
        final_epsilon: float,
        discount_factor: float = 0.95,
    ):
        """Initialize a Reinforcement Learning agent with an empty dictionary
        of state-action values (q_values), a learning rate and an epsilon.

        Args:
            learning_rate: The learning rate
            initial_epsilon: The initial epsilon value
            epsilon_decay: The decay for epsilon
            final_epsilon: The final epsilon value
            discount_factor: The discount factor for computing the Q-value
        """
        self.q_values = defaultdict(lambda: np.zeros(env.action_space.n))

        self.lr = learning_rate
        self.discount_factor = discount_factor

        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon

        self.training_error = []

    def get_action(self, env, obs: tuple[int, int, bool]) -> int:
        """
        Returns the best action with probability (1 - epsilon)
        otherwise a random action with probability epsilon to ensure exploration.
        """
        # with probability epsilon return a random action to explore the environment
        if np.random.random() < self.epsilon:
            return env.action_space.sample()

        # with probability (1 - epsilon) act greedily (exploit)
        else:
            return int(np.argmax(self.q_values[obs]))

    def update(
        self,
        obs: tuple[int, int, bool],
        action: int,
        reward: float,
        terminated: bool,
        next_obs: tuple[int, int, bool],
    ):
        """Updates the Q-value of an action."""
        future_q_value = (not terminated) * np.max(self.q_values[next_obs])
        temporal_difference = (
            reward + self.discount_factor * future_q_value - self.q_values[obs][action]
        )

        self.q_values[obs][action] = (
            self.q_values[obs][action] + self.lr * temporal_difference
        )
        self.training_error.append(temporal_difference)

    def decay_epsilon(self):
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)

In [ ]:
# hyperparameters
learning_rate = 0.01
start_epsilon = 1.0
epsilon_decay = start_epsilon / (n_episodes / 2)  # reduce the exploration over time
final_epsilon = 0.1

agent = BlackjackAgent(
    env=env,
    learning_rate=learning_rate,
    initial_epsilon=start_epsilon,
    epsilon_decay=epsilon_decay,
    final_epsilon=final_epsilon,
)

In [ ]:
def train(agent, env):
    best_reward = -float('inf')
    for episode in tqdm(range(n_episodes)):
        done = False
        obs = env.reset()
        ### printing the obs for debugging
        #print(obs, ' obs')
        #print(type(obs), ' obs type')
        #print(obs[0][0], ' obs[0]')
        #print(obs[0][1], ' obs[1]')
        #print(obs[1], ' obs[1]')
        #print(type(obs[0]), ' obs[0] type')
        #print(type(obs[1]), ' obs[1] type')
        total_reward = 0.0
        while not done:
            action = agent.get_action(env, obs[0])
            next_obs, reward, terminated, truncated, info = env.step(action)

            agent.update(obs[0], action, reward, terminated, next_obs[0])
            done = terminated or truncated

            obs = next_obs
            total_reward += reward
        if total_reward > best_reward:
            best_reward = total_reward
        print("Episode#:{} reward:{} best_reward:{} eps:{}".format(episode,
                                     total_reward, best_reward, agent.epsilon))
        agent.decay_epsilon()
    # Return the trained policy

    return agent.q_values

In [ ]:
learned_policy = train(agent, env)

  0%|          | 179/50000 [00:00<00:55, 893.99it/s]

Episode#:0 reward:-1.0 best_reward:-1.0 eps:1.0
Episode#:1 reward:1.0 best_reward:1.0 eps:0.99996
Episode#:2 reward:-1.0 best_reward:1.0 eps:0.9999199999999999
Episode#:3 reward:-1.0 best_reward:1.0 eps:0.9998799999999999
Episode#:4 reward:-1.0 best_reward:1.0 eps:0.9998399999999998
Episode#:5 reward:-1.0 best_reward:1.0 eps:0.9997999999999998
Episode#:6 reward:1.0 best_reward:1.0 eps:0.9997599999999998
Episode#:7 reward:1.0 best_reward:1.0 eps:0.9997199999999997
Episode#:8 reward:1.0 best_reward:1.0 eps:0.9996799999999997
Episode#:9 reward:1.0 best_reward:1.0 eps:0.9996399999999996
Episode#:10 reward:-1.0 best_reward:1.0 eps:0.9995999999999996
Episode#:11 reward:-1.0 best_reward:1.0 eps:0.9995599999999996
Episode#:12 reward:-1.0 best_reward:1.0 eps:0.9995199999999995
Episode#:13 reward:1.0 best_reward:1.0 eps:0.9994799999999995
Episode#:14 reward:-1.0 best_reward:1.0 eps:0.9994399999999994
Episode#:15 reward:-1.0 best_reward:1.0 eps:0.9993999999999994
Episode#:16 reward:-1.0 best_rewa

  1%|          | 372/50000 [00:00<00:52, 940.18it/s]

Episode#:179 reward:1.0 best_reward:1.0 eps:0.9928399999999928
Episode#:180 reward:-1.0 best_reward:1.0 eps:0.9927999999999928
Episode#:181 reward:1.0 best_reward:1.0 eps:0.9927599999999928
Episode#:182 reward:-1.0 best_reward:1.0 eps:0.9927199999999927
Episode#:183 reward:1.0 best_reward:1.0 eps:0.9926799999999927
Episode#:184 reward:-1.0 best_reward:1.0 eps:0.9926399999999926
Episode#:185 reward:-1.0 best_reward:1.0 eps:0.9925999999999926
Episode#:186 reward:-1.0 best_reward:1.0 eps:0.9925599999999926
Episode#:187 reward:-1.0 best_reward:1.0 eps:0.9925199999999925
Episode#:188 reward:-1.0 best_reward:1.0 eps:0.9924799999999925
Episode#:189 reward:-1.0 best_reward:1.0 eps:0.9924399999999924
Episode#:190 reward:1.0 best_reward:1.0 eps:0.9923999999999924
Episode#:191 reward:1.0 best_reward:1.0 eps:0.9923599999999924
Episode#:192 reward:1.0 best_reward:1.0 eps:0.9923199999999923
Episode#:193 reward:-1.0 best_reward:1.0 eps:0.9922799999999923
Episode#:194 reward:-1.0 best_reward:1.0 eps:0

  1%|          | 569/50000 [00:00<00:51, 963.34it/s]

...

Episode#:49562 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49563 reward:0.0 best_reward:1.0 eps:0.1
Episode#:49564 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49565 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49566 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49567 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49568 reward:0.0 best_reward:1.0 eps:0.1
Episode#:49569 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49570 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49571 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49572 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49573 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49574 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49575 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49576 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49577 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49578 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49579 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49580 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49581 reward:-1.0 best_reward:1.0

100%|█████████▉| 49834/50000 [00:56<00:00, 827.50it/s]


Episode#:49738 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49739 reward:0.0 best_reward:1.0 eps:0.1
Episode#:49740 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49741 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49742 reward:0.0 best_reward:1.0 eps:0.1
Episode#:49743 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49744 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49745 reward:0.0 best_reward:1.0 eps:0.1
Episode#:49746 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49747 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49748 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49749 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49750 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49751 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49752 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49753 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49754 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49755 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49756 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49757 reward:1.0 best_reward:1.0 e

100%|██████████| 50000/50000 [00:56<00:00, 887.44it/s]


Episode#:49897 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49898 reward:0.0 best_reward:1.0 eps:0.1
Episode#:49899 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49900 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49901 reward:0.0 best_reward:1.0 eps:0.1
Episode#:49902 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49903 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49904 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49905 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49906 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49907 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49908 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49909 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49910 reward:0.0 best_reward:1.0 eps:0.1
Episode#:49911 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49912 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49913 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49914 reward:-1.0 best_reward:1.0 eps:0.1
Episode#:49915 reward:1.0 best_reward:1.0 eps:0.1
Episode#:49916 reward:1.0 best_reward:1.0 

In [ ]:
def test(agent, env, policy):
    done = False
    obs = env.reset()
    total_reward = 0.0
    while not done:
        print(obs[0], ' obs')

        action = np.argmax(policy[obs[0]])
        next_obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        obs = next_obs
        total_reward += reward
    return total_reward

In [ ]:
# Uses the Gym Monitor wrapper to evalaute the agent and record video
# only one video will be saved

# video of the final episode with the episode trigger
env = gym.wrappers.RecordVideo(
    env, "./gym_monitor_output", episode_trigger=lambda x: x == 0)

test(agent, env, learned_policy)

env.close()

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /content/gym_monitor_output folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


(10, 3, 0)  obs
20  obs
Moviepy - Building video /content/gym_monitor_output/rl-video-episode-0.mp4.
Moviepy - Writing video /content/gym_monitor_output/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/gym_monitor_output/rl-video-episode-0.mp4
